In [1]:
import pandas as pd
from xml.etree import ElementTree
import requests
import pubmed_xml

### Script to pull files from pubmed

In [6]:
gad_df = pd.read_csv('../dataset/GADCDC_data.tsv', sep='\t', usecols=[0, 1, 2, 3, 5, 9], dtype=str)
gad_df.dropna(subset=['DISEASE', 'GENE', 'PUBMEDID'], inplace=True)

In [7]:
pubmed_id = pd.Series(gad_df['PUBMEDID'].unique())

In [4]:
articles = ','.join(list(pubmed_id[:200]))

In [2]:
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'

In [3]:
r = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=7593547&retmode=xml')

In [94]:
r = requests.get(url = base_url, params = {'db':'pubmed', 'retmode':'xml', 'id':'16120569'})

In [1]:
files = pubmed_xml.many_files('abstract', list(pubmed_id))

In [4]:
pubmed_xml.write_files('abstract', files)

In [8]:
len(list(pubmed_id))

54070

In [68]:
tree = ElementTree.fromstring(r.text)
for child in tree:
    pubmed_id = child[0][0].text
    ElementTree.ElementTree(child).write('../corpus/' + pubmed_id + '.xml')

In [ ]:
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&rettype=abstract&id=15304010'

#### PUBMED will not let you make automated requests
#### There are 62 GB of tar.gz files on ftp which is not feasable to go through
#### able to use the Entrez Utilities api to make requests
#### Limit of 3/10 (w/ api key) requests per second (we can request more if needed)
##### able to get around this: https://www.ncbi.nlm.nih.gov/books/NBK25498/#chapter3.Application_3_Retrieving_large

In [ ]:
# making the Post request https://eutils.ncbi.nlm.nih.gov/entrez/eutils/epost.fcgi
# looks like this https://eutils.ncbi.nlm.nih.gov/entrez/eutils/epost.fcgi?db=pubmed&id=11237011,12466850
# base url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/
# post service epost.fcgi
# db=pubmed |use the pubmed database
# id=XXXXX,XXXXX,.... The ids to upload
# Note there is no maximum for ids to post if more than 200 should be made using the HTTP POST method
# possible to do this over multiple requests -> posts create WebEnv specify this with: WebEnv=<Some String>
# If WebEnv is not provided, creates a new one (returned somehow) the ID list is saved to query_key 1
# it might be the case that with multiple calls to the same WebEnv that each set is added to new query key?
# ----------------------------------------------------------------------
# retrieving this
# base_url(above) +
# efetch.fcgi | efetch is for fetching data
# db=pubmed | want results from pubmed
# rettype=abstract&retmode=text | want only the abstract
# retmode=xml | want the result as xml for whole entry
#---------------------------------
# If using epost
# WebEnv=<Some String> | From the epost above
# query_key=<Some Key> | from the epost above
# retstart=<Starting Point> |  0 indexed
# retmax=<Some Amount> | max of 10,000
# ----------------------------------
# Id providing list of IDs
# id = XXXX,XXXX,XXXX.... max of 200 with HTTP GET? need to use HTTP Post?
# POST 
# need to create an http user agent
# make an http request object on the default efetch url
# set the tontent type to 'application/x-www-form-urlencoded'
# set the content to url params the ?X=Y&X=Y... above
# May need to set ids like id=XX&id=XX&id=XX...
# using the agent, make the request
# need to do this in batches of 10,000?